## Импорт библиотек

In [64]:
import pandas as pd
import os
import warnings
import numpy as np
import re
import json

from functools import reduce
from datetime import datetime
from datetime import date

warnings.filterwarnings("ignore")

## Определение функций

## Установка параметров

In [2]:
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
                     'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
data_columns = ['bodyType', 'brand', 'car_url', 'color', 'engineDisplacement', 'enginePower', 
                'equipment_dict','fuelType', 'mileage', 'modelDate', 'model_name', 'numberOfDoors', 
                'productionDate', 'sell_id', 'vehicleTransmission', 'vendor', 
                'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль']
outer_datasets = ['all_auto_ru_09_09_2020.csv',
                     'all_car_27_08_2021_data_12_brands.csv',
                     'data1.csv',
                     'parsing_auto_ru.csv',
                     'train_df_full_part1.csv']

### Сокращение (перезапись только брендов из test)

In [3]:
# for df_name,pars_date in  zip(outer_datasets,parsing_date):
#     try:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#     except Exception:
#         df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#     df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
#     file_name = 'auto_ru_' + pars_date + '.csv'
#     print(f'./Project_7_data/{file_name}')
#     df.to_csv(f'./Project_7_data/{file_name}',index=False)

### Определение даты каждого датсета/ курса на эту дату

In [4]:
parsing_date = ['2020_09_09','2021_08_29','2020_10_19','2021_08_13','2021_10_01']
currency_rate = [1,1,1,1,1]
basket_currency_rate_ = [1,1,1,1,1]

In [5]:
datasets_to_train = []
for df_name,pars_date in  zip(outer_datasets,parsing_date):
    file_name = 'auto_ru_' + pars_date + '.csv'
    datasets_to_train.append(file_name)

### Корреспонденция колонок

### унификация с test  - auto_ru_2020_09_09.csv  с учетом data_columns

In [6]:
test = pd.read_csv('./Project_7_data/test.csv',low_memory=False)

In [7]:
df= pd.read_csv('./Project_7_data/' + datasets_to_train[0],low_memory=False)

In [8]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [9]:
df.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')

## ============================================

#### bodyType

In [10]:
def bodyType_unification(x):
    res=[]
    x = x.lower() if type(x) == str else x  # <================
    try:
        for body_type in test.bodyType.unique():
            if body_type in x:
                res.append(body_type)
    except Exception: return x
    return max(res)

In [11]:
%%time
df.dropna(subset=['bodyType'],inplace=True)

lambda x: x.lower() if x else x
df.bodyType = df.bodyType.apply(bodyType_unification)

Wall time: 2min 32s


In [12]:
set(df.bodyType.unique()) == set(test.bodyType.unique())

True

In [123]:
set(df.bodyType.unique())

{'внедорожник 3 дв.',
 'внедорожник 5 дв.',
 'внедорожник открытый',
 'кабриолет',
 'компактвэн',
 'купе',
 'купе-хардтоп',
 'лимузин',
 'лифтбек',
 'микровэн',
 'минивэн',
 'пикап двойная кабина',
 'пикап одинарная кабина',
 'пикап полуторная кабина',
 'родстер',
 'седан',
 'седан 2 дв.',
 'седан-хардтоп',
 'тарга',
 'универсал 5 дв.',
 'фастбек',
 'фургон',
 'хэтчбек 3 дв.',
 'хэтчбек 5 дв.'}

#### brand

In [13]:
set(df.brand.unique()) == set(test.brand.unique())

True

In [122]:
set(df.brand.unique())

{'AUDI',
 'BMW',
 'HONDA',
 'INFINITI',
 'LEXUS',
 'MERCEDES',
 'MITSUBISHI',
 'NISSAN',
 'SKODA',
 'TOYOTA',
 'VOLKSWAGEN',
 'VOLVO'}

#### car_url <======================================================

#### color ----------------------------------- уточнить цвета

In [14]:
color_codes = {
    '040001': 'чёрный',
    'FAFBFB': 'белый', 
    '0000CC': 'синий', 
    '200204': 'коричневый', 
    'EE1D19': 'красный', 
    'CACECB': 'серый',
    'C49648': 'бежевый', 
    '97948F': 'серебристый', 
    'FFD600': 'золотистый', 
    'FF8649': 'оранжевый', 
    '22A0F8': 'голубой',
    'FFC0CB': 'пурпурный', 
    'DEA522': 'жёлтый', 
    '007F00': 'зелёный', 
    '660099': 'фиолетовый',
    '4A2197': 'фиолетовый'}
df.color = df.color.map(color_codes)

In [15]:
set(test.color.unique()) == set(df.color.unique())

False

#### engineDisplacement

In [16]:
test.engineDisplacement = test.engineDisplacement.apply(
    lambda x: float(x[:-4]) if x[:-4] else np.NaN)   

In [40]:
def engineDisplacement_unification(x):
    x = float(re.sub("[^\d.]", r'', x)) if re.sub("[^\d.]", r'', x) else 0
    if x >= 7: x = 0
#     if 10 <= x < 100: return x/10
#     if 100 <= x: return x/100
    return x

df.engineDisplacement = df.engineDisplacement.apply(engineDisplacement_unification)

#### enginePower

In [49]:
test.enginePower = test.enginePower.apply(lambda x: int(x[:-4]))

In [53]:
df.enginePower = df.enginePower.astype(int)

#### equipment_dict

In [114]:
test.equipment_dict = test.equipment_dict.apply(lambda x: list(json.loads(x).keys()) if x==x else [])

In [116]:
def train_options(x):
    point = "'available_options': "
    start = x.find(point)+len(point)+2
    finish = x.find("]",start) - 1
    return x[start:finish].split("', '")

df.Комплектация = df.Комплектация.apply(train_options)

#### fuelType

In [120]:
set(test.fuelType.unique()) == set(df.fuelType.unique())

True

In [121]:
set(test.fuelType.unique())

{'бензин', 'газ', 'гибрид', 'дизель', 'электро'}

#### mileage

#### modelDate

In [130]:
df.modelDate = df.modelDate.astype(int)

array([1990, 1982, 1988, 1976, 1983, 1979, 1986, 1991, 1978, 1987, 1992,
       1938, 2010, 2014, 2000, 2016, 2004, 2012, 2008, 1996, 2003, 2015,
       2011, 2007, 1999, 1994, 2009, 2018, 2001, 1997, 2006, 2019, 2013,
       2017, 2002, 2005, 1980, 1984, 1998, 1966, 1975, 1937, 1936, 1949,
       2020, 1995, 1981, 1972, 1977, 1989, 1968, 1993, 1985, 1973, 1969,
       1934, 1960, 1974, 1950, 1961, 1955, 1965, 1904, 1963, 1971, 1959,
       1958, 1951, 1957])

#### model_name

In [139]:
mn1 = set(test.model_name.unique())

In [140]:
mn2 = set(df.model.unique())

In [141]:
mn2 - mn1

{'180SX',
 'BREVIS',
 'CELSIOR',
 'CLIPPER_RIO',
 'EDIX',
 'EQC',
 'KLUGER',
 'LANCER_CARGO',
 'LIFE',
 'M8',
 'MISTRAL',
 'MOCO',
 'N_ONE',
 'PAO',
 'PASSPORT',
 'PRIMASTAR',
 'Q3_SPORTBACK',
 'RC_F',
 'REGIUSACE',
 'ROOMY',
 'SLC_KLASSE_AMG',
 'SLK_KLASSE_AMG',
 'S_MX',
 'TACOMA',
 'TITAN',
 'TYPE_2',
 'TYPE_3',
 'W110',
 'W189'}

In [142]:
mn1 - mn2

{'300ZX',
 'ARTEON',
 'ATLAS',
 'COROLLA_II',
 'GOLF_R32',
 'IST',
 'LANCER_RALLIART',
 'LM',
 'OTTI',
 'POLO_GTI',
 'PORTE',
 'PRESEA',
 'RSQ3',
 'SPARKY',
 'TORNEO',
 'WILL_CYPHA',
 'X4_M',
 'Z',
 'ZEST'}

#### numberOfDoors

#### productionDate

#### sell_id

#### vehicleTransmission

#### vendor

#### Владельцы

#### Владение

#### ПТС

#### Привод

#### Руль

### Стандартизация датасетов

In [116]:
sorted(test.bodyType.unique())

['внедорожник 3 дв.',
 'внедорожник 5 дв.',
 'внедорожник открытый',
 'кабриолет',
 'компактвэн',
 'купе',
 'купе-хардтоп',
 'лимузин',
 'лифтбек',
 'микровэн',
 'минивэн',
 'пикап двойная кабина',
 'пикап одинарная кабина',
 'пикап полуторная кабина',
 'родстер',
 'седан',
 'седан 2 дв.',
 'седан-хардтоп',
 'тарга',
 'универсал 5 дв.',
 'фастбек',
 'фургон',
 'хэтчбек 3 дв.',
 'хэтчбек 5 дв.']

In [118]:
test.bodyType.value_counts()

седан                      13147
внедорожник 5 дв.          12228
лифтбек                     2232
хэтчбек 5 дв.               2124
универсал 5 дв.             1276
минивэн                      995
купе                         824
компактвэн                   489
хэтчбек 3 дв.                485
пикап двойная кабина         280
купе-хардтоп                 158
внедорожник 3 дв.            115
родстер                       99
фургон                        97
кабриолет                     90
седан-хардтоп                 12
микровэн                       7
лимузин                        6
седан 2 дв.                    6
пикап полуторная кабина        5
пикап одинарная кабина         5
внедорожник открытый           3
тарга                          2
фастбек                        1
Name: bodyType, dtype: int64

### Анализ распределения цены в 5(10) топ моделях

In [22]:
for df_name in  datasets_to_train:
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
    except Exception:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
    df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
    
    print(df.columns)
#     models_for_distribution = df.model_name.value_counts().index[:5].to_list()
#     print(models_for_distribution)

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')
Index(['Unnamed: 0', 'car_url', 'bodyType', 'brand', 'color',
       'complectation_dict', 'description', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage',
       'modelDate', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime',
       'price', 'priceCurrency', 'productionDate', 'sell_id', 'views',
       'date_added', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль',
       'Состояние', 'Таможня', 'region'],
      dtype='object')
Index(['bodyType', 'brand', 'car_url', 'col

In [20]:
models_for_distribution = df.model_name.value_counts().index[:5].to_list()
models_for_distribution

['Tiguan', 'X-Trail', 'Land Cruiser', 'Touareg', 'Polo']

### Соотношение курсового коэфициента с рапределением

In [10]:
'''for df_name in  datasets_to_train:
    
    try:
        df = pd.read_csv('./Preproject_7_data/' + df_name,low_memory=False)
#         print('ok')
    except:
        df = pd.read_csv('./Preproject_7_data/' + df_name,sep='\t',low_memory=False) 
#         print('trouble')
#     print(df.brand.unique())
    df = df[df.brand.apply(lambda x:x in marks_for_parsing)]
#     print(df.brand.unique())
    print(df_name,end=' ')
    try:
        print(date.fromtimestamp((df.parsing_unixtime.min())))
    except:
        print('no parsing_unixtime')
    print(df_name,end=' ')
    try:
        print(date.fromtimestamp((df.parsing_unixtime.max())))
    except:
        print('no parsing_unixtime')''';

## ==================================

In [27]:
def trfn(x):
    try:res = x.lower()   
    except Exception:res = x
    return res

df.bodyType.apply(trfn).unique()[:15]

array(['седан', 'универсал 5 дв.', 'хэтчбек 5 дв. sportback',
       'хэтчбек 3 дв.', 'хэтчбек 5 дв.', 'кабриолет', 'купе',
       'лифтбек sportback', 'лифтбек', 'седан long', 'внедорожник 5 дв.',
       'кабриолет roadster', 'седан 2 дв.', 'седан gran coupe',
       'компактвэн'], dtype=object)